<h1 style="text-align: center;">Periodic Boundary Conditions (chameleon)</h1>

This notebook shows how to use periodic boundary conditions (epbc) with *femtoscope*. This type of boundary condition is useful e.g. in the study of various materials, because one only has to model a finite representative volume element. Here, we illustrate this feature on the chameleon Klein-Gordon equation.

*prerequisites* :
- FEM & periodic BC knowledge
- `mesh_generation_basics` notebook
- `nonlinear_problems` notebook

**If you have questions/comments or want to report a bug, please send me an email at <a href="mailto:hugo.levy@onera.fr">hugo.levy@onera.fr</a>**

In [1]:
# Add femtoscope to the path
%reset
%matplotlib inline
import sys
sys.path.append("../")  # go to parent dir

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [11]:
%%capture

# Module imports
import numpy as np
import matplotlib.pyplot as plt

import femtoscope
from femtoscope.inout.meshfactory import generate_mesh_from_geo
from femtoscope.physics.physical_problems import Chameleon

## Mesh creation

Periodic boundary conditions in femtoscope come with two requirements on the underlying mesh:
1) the numerical domain must be a box (rectangle in 2D, rectangular prism in 3D);
2) opposite sides must possess the same surface mesh up to a translation.

The second condition can be imposed in Gmsh by using the commands `Periodic Curve` or `Periodic Surface` in the `.geo` script. Here we use the `cube_periodic.geo` script. Note that the boundary faces are not included in the physical groups, as they will be selected through functions at a later stage. This is because corner nodes have to belong to several faces simultaneously, which cannot be achieved with physical groups unfortunately.

In [7]:
# Mesh creation
pre_mesh = generate_mesh_from_geo("cube_periodic", param_dict={'size': 0.4}, show_mesh=True)

## Chameleon problem

In [8]:
# Physical parameters
alpha = 0.1  # chameleon dimensionless parameter
npot = 1  # chameleon exponent
rho_min = 0.1  # kg/m^3
rho_max = 10  # kg/m^3
phi_min = rho_max ** (-1 / (npot + 1))
phi_max = rho_min ** (-1 / (npot + 1))
param_dict = {'alpha': alpha, 'npot': npot, 'rho_min': rho_min, 'rho_max': rho_max}

# FEM parameters
fem_order = 1
dim = 3
coorsys = 'cartesian'

# Entitiy selection
def xface1(coors, domain=None):
    return np.where(abs(coors[:, 0] - (-0.5)) < 1e-8)[0]

def xface2(coors, domain=None):
    return np.where(abs(coors[:, 0] - 0.5) < 1e-8)[0]

def yface1(coors, domain=None):
    return np.where(abs(coors[:, 1] - (-0.5)) < 1e-8)[0]

def yface2(coors, domain=None):
    return np.where(abs(coors[:, 1] - 0.5) < 1e-8)[0]

def zface1(coors, domain=None):
    return np.where(abs(coors[:, 2] - (-0.5)) < 1e-8)[0]

def zface2(coors, domain=None):
    return np.where(abs(coors[:, 2] - 0.5) < 1e-8)[0]

dim_func_entities = [(2, xface1, 200), (2, xface2, 201),
                     (2, yface1, 202), (2, yface2, 203),
                     (2, zface1, 204), (2, zface2, 205)]

# Density assignement
density_dict = {('subomega', 300): rho_max, ('subomega', 301): rho_min}

# EPBC
pre_epbc_list = [[('facet', 200), ('facet', 201), 'match_x_plane'],
                 [('facet', 202), ('facet', 203), 'match_y_plane'],
                 [('facet', 204), ('facet', 205), 'match_z_plane']]

# Chameleon problem
chameleon = Chameleon(param_dict, dim, coorsys=coorsys)
partial_args_dict = {
    'dim': dim,
    'name': 'wf_per',
    'pre_mesh': pre_mesh,
    'fem_order': fem_order,
    'dim_func_entities': dim_func_entities,
    'pre_epbc_list': pre_epbc_list
}
chameleon.set_wf_int(partial_args_dict, density_dict)
chameleon.set_wf_residual(partial_args_dict, density_dict)
chameleon.set_default_solver()
chameleon.set_default_monitor(50)

In [9]:
solver = chameleon.default_solver
solver.solve()

_________________________ MONITORING PARAMETERS __________________________
                 criterion            look          active       threshold
         MaximumIterations            True            True        5.00e+01
RelativeDeltaSolutionNorm2            True            True        1.00e-08
            ResidualVector            True           False       -1.00e+00
       ResidualVectorNorm2            True           False       -1.00e+00
   ResidualReductionFactor            True           False       -1.00e+00
       ResidualVectorParts            True           False       -1.00e+00
--------------------------------------------------------------------------

_____________________ ITERATION NO 1 _____________________
                 criterion           value       threshold
         MaximumIterations        1.00e+00        5.00e+01
RelativeDeltaSolutionNorm2        7.20e-01        1.00e-08
       ResidualVectorNorm2        6.57e-02       -1.00e+00
   ResidualReductionFactor   

As shown in the other tutorials (especially the `offline_postprocessing` notebook), the results of this FEM computation can be saved to a .vtk file. Then, one can use third-party tools such as `paraview` to inspect the solution, *femtoscope* having no built-in routines to inspect 3-dimensional data.

In [12]:
# Save the results to a .vtk file
solver.save_results_to_vtk('chameleon_periodic_tutorial')  # file saved to TMP_DIR
print(femtoscope.TMP_DIR)

D:\hlevy\Documents\femtoscope\data\tmp
